# Logging Service Tests

Dieses Notebook testet alle 3 Funktionen aus `backend.logging_service`:
- `log_request()` — Einzelnen Eintrag schreiben
- `read_logs()` — Alle Einträge lesen
- `get_stats()` — Statistiken berechnen

In [3]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from pathlib import Path
from backend.logging_service import log_request, read_logs, get_stats
import backend.logging_service as logging_service

# Umbiegen auf Test-Logdatei, damit echte Logs sauber bleiben
logging_service.LOG_FILE = logging_service.LOGS_DIR / "test_requests.jsonl"

# Sicherstellen dass die Test-Datei leer startet
if logging_service.LOG_FILE.exists():
    logging_service.LOG_FILE.unlink()

print(f"LOG_FILE: {logging_service.LOG_FILE}")
print("Setup OK")

LOG_FILE: /Users/dennis/Documents/Developer/BudgetRouter/ai-model-api-budget-router/logs/test_requests.jsonl
Setup OK


## 1. log_request — Einzelnen Eintrag schreiben

In [4]:
# Einen Beispiel-Request loggen
log_request({"model": "llama-3.3-70b-versatile", "actual_cost": 0.001, "task_type": "general"})

# Log-Datei direkt lesen und prüfen
content = logging_service.LOG_FILE.read_text()
print("Inhalt der Log-Datei:")
print(content)


Inhalt der Log-Datei:
{"timestamp": "2026-02-13T18:47:03.604031+00:00", "model": "llama-3.3-70b-versatile", "actual_cost": 0.001, "task_type": "general"}



## 2. read_logs — Alle Einträge lesen

In [5]:
# Weitere Einträge loggen (verschiedene Modelle und Kosten)
log_request({"model": "llama-3.1-8b-instant", "actual_cost": 0.0002, "task_type": "email"})
log_request({"model": "llama-3.3-70b-versatile", "actual_cost": 0.0015, "task_type": "code"})
log_request({"model": "gemma2-9b-it", "actual_cost": 0.0003, "task_type": "summarize"})

# Alle Logs lesen
logs = read_logs()
print(f"Anzahl Einträge: {len(logs)}")
assert len(logs) == 4, f"Erwartet: 4, Bekommen: {len(logs)}"

print(f"\n{'#':<4} {'Model':<30} {'Cost':>10} {'Timestamp':<28}")
print("-" * 75)
for i, log in enumerate(logs):
    assert "timestamp" in log, f"Eintrag {i} hat keinen Timestamp!"
    print(f"{i+1:<4} {log['model']:<30} {log['actual_cost']:>10.4f} {log['timestamp']:<28}")

print("\nread_logs OK")

Anzahl Einträge: 4

#    Model                                Cost Timestamp                   
---------------------------------------------------------------------------
1    llama-3.3-70b-versatile            0.0010 2026-02-13T18:47:03.604031+00:00
2    llama-3.1-8b-instant               0.0002 2026-02-13T18:47:48.038705+00:00
3    llama-3.3-70b-versatile            0.0015 2026-02-13T18:47:48.039409+00:00
4    gemma2-9b-it                       0.0003 2026-02-13T18:47:48.039712+00:00

read_logs OK


## 3. get_stats — Statistiken berechnen

In [6]:
stats = get_stats()

print("Statistiken:")
print(f"  Total Requests:  {stats['total_requests']}")
print(f"  Total Cost:      ${stats['total_cost']:.6f}")
print(f"  Average Cost:    ${stats['average_cost']:.6f}")
print(f"  Model Usage:     {stats['model_usage']}")

# Prüfungen
assert stats["total_requests"] == 4
expected_total = 0.001 + 0.0002 + 0.0015 + 0.0003
assert abs(stats["total_cost"] - expected_total) < 1e-6, f"Total cost falsch: {stats['total_cost']} != {expected_total}"
assert abs(stats["average_cost"] - expected_total / 4) < 1e-6
assert stats["model_usage"]["llama-3.3-70b-versatile"] == 2
assert stats["model_usage"]["llama-3.1-8b-instant"] == 1
assert stats["model_usage"]["gemma2-9b-it"] == 1

print("\nget_stats OK")

Statistiken:
  Total Requests:  4
  Total Cost:      $0.003000
  Average Cost:    $0.000750
  Model Usage:     {'llama-3.3-70b-versatile': 2, 'llama-3.1-8b-instant': 1, 'gemma2-9b-it': 1}

get_stats OK


## 4. Cleanup — Test-Logdatei löschen

In [ ]:
# Test-Logdatei löschen
logging_service.LOG_FILE.unlink(missing_ok=True)
print(f"Test-Datei gelöscht: {not logging_service.LOG_FILE.exists()}")

# Prüfen dass echte Logs unberührt sind
real_log = logging_service.LOGS_DIR / "requests.jsonl"
print(f"Echte Log-Datei existiert: {real_log.exists()}")

# LOG_FILE auf Originalwert zurücksetzen
logging_service.LOG_FILE = real_log
print(f"LOG_FILE zurückgesetzt auf: {logging_service.LOG_FILE}")

print("\nCleanup OK — alle Tests bestanden!")

TypeError: Path.unlink() got an unexpected keyword argument 'exist_ok'